In [64]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import re
import functools
from tqdm.notebook import tqdm
import subprocess
import glob
from IPython import display as ipythondisplay

In [57]:
def make_datapath_list(rootpath):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    

    path_list = []
    
    for c in tqdm(alphabet):
        target_path = os.path.join(rootpath + c + '/*.abc')
        for path in glob.glob(target_path):
            path_list.append(path)

    return path_list

In [35]:
def extract_song_data(text):
    # extract song from abc notation
    # remove header
    text = re.sub(r'%%.*', '', text)
    text = text.strip()
    text = re.sub(r'\n{2,}',r'\n',text)
    return text

In [ ]:
def save_song_to_abc(song, filename):
    save_name = "/result/{}.abc".format(filename)
    with open(save_name, 'w') as f:
        f.write(song)
    
    return filename

def abc2wav(abc_file):
    suf = abc_file.rsrip('.abc')
    cmd = "abc2midi {} -o {}".format(abc_file, suf + ".mid")
    os.system(cmd)
    cmd = "timidity {}.mid -Ow {}.wav".format(suf, suf)

    return os.system()

def play_wav(wav_file):
    return ipythondisplay.Audio(wav_file)

def play_song(song):
    basename = save_song_to_abc(song)
    ret = abc2wav(basename + ".abc")
    if ret == 0:
        return play_wav(basename + ".wav")
    
    return None

In [58]:
cwd = os.getcwd()
rootpath = cwd + '/notation/'
notation_list = make_datapath_list(rootpath)

In [61]:
songs = []

for notation in tqdm(notation_list):
    with open(notation, 'r') as f:
        text = f.read()
        song = extract_song_data(text)
        songs.append(song)

  0%|          | 0/1744 [00:00<?, ?it/s]

In [63]:
example_song = songs[0]
print("Example song: ")
print(example_song)

Example song: 
X: 0
T: Angelus Ad Virginum
Z: Public Domain
L: 1/4
M: 6/8
Q: 1/4=120
V: P1 name="melody"
V: P2 name="drone"
K: C
[V: P1]  d B/ d d/ | c B/ A A/ | B G/ A/G/F/ | G3/ G3/ | d B/ d d/ | c B/ A A/ | B G/ A/G/F/ | G3/ G3/ | [M: 9/8]  A3/ c c/ B G/ | [M: 6/8]  A/B/A/ G3/ | [M: 9/8]  A3/ c c/ B G/ | A/G/A/ D3/ E/ ^F | [M: 6/8]  G F/ E D/ | G D/ G B/ | B/A/G/ A D/ | E F/ E/D/C/ | D3/ D3/|]
[V: P2]  [G,D] [G,/D/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,3/D3/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,3/D3/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [M: 9/8]  [G,3/D3/] [G,3/D3/] [G,3/D3/] | [M: 6/8]  [G,D] [G,/D/] [G,3/D3/] | [M: 9/8]  [G,3/D3/] [G,D] [G,/D/] [G,D] [G,/D/] | [G,D] [G,/D/] [G,3/D3/] [G,3/D3/] | [M: 6/8]  [G,3/D3/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/] | [G,D] [G,/D/] [G,3/D3/]|]
